In [1]:
import itertools as itt
import math
from collections import Counter, defaultdict

import bioregistry
import bioversions
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import pyobo
import seaborn as sns
from disease import CONFIGURATION, MODULE
from IPython.display import SVG
from pyobo.sources.mesh import get_mesh_category_curies
from tqdm.auto import tqdm

import semra
import semra.api
from semra.api import get_index, to_multidigraph
from semra.io import from_pickle, from_sssom

/Users/cthoyt/.virtualenvs/indra/lib/python3.11/site-packages/pydantic/_internal/_config.py:322: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


## Summarize the Resources

In [2]:
# some resources are generic, so we want to cut to a relevant subset
subsets = {
    "mesh": [*get_mesh_category_curies("C"), *get_mesh_category_curies("F")],
    "efo": ["efo:0000408"],
    "ncit": ["ncit:C2991"],
}

terms = {}
for prefix in CONFIGURATION.priority:
    id_name_mapping = pyobo.get_id_name_mapping(prefix)
    print(prefix, len(id_name_mapping))
    subset = {
        descendant
        for parent_curie in subsets.get(prefix, [])
        for descendant in pyobo.get_descendants(parent_curie) or []
    }
    if subset:
        terms[prefix] = {k: v for k, v in id_name_mapping.items() if f"{prefix}:{k}" in subset}
    else:
        terms[prefix] = id_name_mapping

doid 11511
mondo 24099
efo 16410
mesh 354069
ncit 181982
orphanet 0


[orphanet.ordo] parsing obo: 0.00it [00:00, ?it/s]

[orphanet.ordo] ontology=https://www.orphadata.com/data/ontologies/ordo/last_version/ORDO_en_4.4.owl has a strange format. replacing with prefix


orphanet.ordo 0
omim 0
omim.ps 0
gard 0
icd10 2345
icd10cm 0
icd10pcs 0
There are 2 top level entities


[icd11 v2024-01] could not load: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Traceback (most recent call last):
  File "/Users/cthoyt/.virtualenvs/indra/lib/python3.11/site-packages/urllib3/connectionpool.py", line 793, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "/Users/cthoyt/.virtualenvs/indra/lib/python3.11/site-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/cthoyt/.virtualenvs/indra/lib/python3.11/site-packages/urllib3/connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_2/Frameworks/Python.framework/Versions/3.11/lib/python3.11/http/client.py", line 1386, in getresponse
    response.begin()
  File "/opt/homebrew/Cellar/python@3.11/3.11.7_2/Frameworks/Python.framework

icd11 0
icd9 0
icd9cm 0


[icdo vNone] could not load: Command '['java', '-jar', '/Users/cthoyt/.data/robot/robot.jar', 'merge', '-i', '/Users/cthoyt/.data/pyobo/raw/icdo/icdo.owl', 'convert', '-o', '/Users/cthoyt/.data/pyobo/raw/icdo/icdo.obo']' returned non-zero exit status 1.
Traceback (most recent call last):
  File "/Users/cthoyt/dev/pyobo/src/pyobo/api/names.py", line 142, in get_id_name_mapping
    return _get_id_name_mapping()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cthoyt/dev/pystow/src/pystow/cache.py", line 83, in _wrapped
    rv = func()
         ^^^^^^
  File "/Users/cthoyt/dev/pyobo/src/pyobo/api/names.py", line 138, in _get_id_name_mapping
    ontology = get_ontology(prefix, force=force, strict=strict, version=version)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cthoyt/dev/pyobo/src/pyobo/identifier_utils.py", line 115, in _wrapped
    return f(norm_prefix, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/cthoy

icdo 0


In [3]:
no_versions = {"omim.ps", "icd10", "icd9", "icd11", "icd10cm", "icd10pcs", "icd9cm", "icdo", "gard", "omim"}
summary_rows = [
    (
        prefix,
        bioregistry.get_license(prefix),
        bioversions.get_version(prefix) if prefix not in no_versions else None,
        len(terms.get(prefix, [])),
    )
    for prefix in CONFIGURATION.priority
]
summary_df = pd.DataFrame(summary_rows, columns=["prefix", "license", "version", "terms"])
summary_df

KeyError: 'orphanet'

## Summarize the Mappings

In [ ]:
mappings = from_pickle(MODULE.join(name="processed.pkl"))
processed_n = len(mappings)
processed_n

In [ ]:
# Remove many-to-many mappings since it overinflates
# mappings = semra.api.filter_many_to_many(mappings, progress=False)
# filtered_n = len(mappings)
# filtered_n

# TODO filter out many-to-many mappings before starting -
# using the typical function doesn't work since there is

In [ ]:
index = get_index(mappings, progress=False)

In [ ]:
directed = defaultdict(set)
for s, p, o in index:
    if p != semra.EXACT_MATCH:
        continue
    directed[s.prefix, o.prefix].add(s.identifier)
    directed[o.prefix, s.prefix].add(o.identifier)

### Asymmetric Summarization

The following summary looks for each ordered pair of resources, what percentage of each resources' terms are mapped to the other resource. Because each resource is a different size, this is an asymmetric measurement.

The way to read this table is the horizontal index corresponds to the source prefix and the columns correspond to the target prefix. This means in the row with label "efo" and column with label "mesh" that has 14% means that 14% of EFO can be mapped to MeSH.

In [ ]:
def _df(xx, default=1.0):
    rows = [
        [default if p1 == p2 else xx.get((p1, p2), None) for p2 in CONFIGURATION.priority]
        for p1 in CONFIGURATION.priority
    ]
    df = pd.DataFrame(rows, columns=CONFIGURATION.priority, index=CONFIGURATION.priority)
    df.index.name = "source_prefix"
    df.columns.name = "target_prefix"
    return df

In [ ]:
asymmetric_counts = {}
for (l, r), l_entities in directed.items():
    l_terms = terms.get(l)
    if l_terms:
        count = len(l_entities.intersection(l_terms))
    else:
        count = len(l_entities)
    asymmetric_counts[l, r] = count

asymmetric_counts_summary_df = _df(asymmetric_counts, default=0).fillna(0).astype(int)
asymmetric_counts_summary_df

In [ ]:
def _graph_svg(counter, scaling_factor: float = 3.0, fmt=",", cls=nx.DiGraph, cutoff=0.0):
    graph = cls()
    for (s, t), count in counter.items():
        if not count:
            continue
        if count <= cutoff:
            continue
        graph.add_edge(s, t, label=f"{count:{fmt}}")

    agraph = nx.nx_agraph.to_agraph(graph)
    agraph.graph_attr["rankdir"] = "LR"

    values = [v for v in counter.values() if v is not None and v > 0]
    bottom, top = min(values), max(values)
    rr = top - bottom

    for edge, weight in counter.items():
        if not weight:
            continue
        x = 1 + scaling_factor * (weight - bottom) / rr
        if agraph.has_edge(*edge):
            agraph.get_edge(*edge).attr["penwidth"] = x
    return agraph.draw(prog="dot", format="svg")


SVG(_graph_svg(asymmetric_counts, cutoff=5))

In [ ]:
asymmetric = {}
for (l, r), count in asymmetric_counts.items():
    denominator = len(terms.get(l, []))
    asymmetric[l, r] = count / denominator if denominator > 0 else None

asymmetric_summary_df = _df(asymmetric)
(asymmetric_summary_df * 100).round(2)

In [ ]:
SVG(_graph_svg(asymmetric, fmt=".1%"))

### Symmetric Summarization

In [ ]:
symmetric_counts = {}
for l, r in directed:
    left = directed[l, r]
    if left_terms := terms.get(l, []):
        left.intersection_update(left_terms)
    right = directed[r, l]
    if right_terms := terms.get(r, []):
        right.intersection_update(right_terms)
    symmetric_counts[l, r] = max(len(left), len(right))
_df(symmetric_counts, default=0).fillna(0).astype(int)

In [ ]:
x = _graph_svg(symmetric_counts, cls=nx.Graph, cutoff=20)
with open("disease_graph.svg", "wb") as file:
    file.write(x)
SVG(x)

In [ ]:
# intersect with the terms for each to make sure we're not keeping any mappings that are irrelevant

symmetric = {}
for (l, r), count in symmetric_counts.items():
    if terms[r] and terms[l]:
        denom = min(len(terms[r]), len(terms[l]))
    elif terms[r]:
        denom = len(terms[r])
    elif terms[l]:
        denom = len(terms[l])
    else:
        denom = 1
    symmetric[l, r] = count / denom

symmetric_df = _df(symmetric).fillna(0.0)

# clip since there might be some artifacts of mappings to terms that don't exist anymore
(symmetric_df * 100).round(4)

In [ ]:
SVG(_graph_svg(symmetric, cls=nx.Graph, fmt=".2%", cutoff=0.01))

# Landscape Coverage

Before, we looked at the overlaps between each resource. Now, we use that information jointly to estimate the number of terms in the landscape itself, and estimate how much of the landscape each resource covers.

In [ ]:
entities = defaultdict(set)
for mapping in mappings:
    for r in (mapping.s, mapping.o):
        entities[r.prefix].add(r.identifier)

counter = Counter()
graph = to_multidigraph(mappings)
for component in nx.weakly_connected_components(graph):
    prefixes = frozenset(r.prefix for r in component)
    counter[prefixes] += 1

at_least_1_mapping = sum(counter.values())

unique_to_single = defaultdict(set)
for prefix in CONFIGURATION.priority:
    prefix_terms = terms.get(prefix)
    if not prefix_terms:
        continue

    prefix_terms = set(prefix_terms)
    mapped_terms = entities[prefix]
    counter[frozenset([prefix])] = unique_to_single[prefix] = len(prefix_terms - mapped_terms)

only_1_mapping = sum(unique_to_single.values())
conserved = counter[frozenset(CONFIGURATION.priority)]
total_entity_estimate = sum(counter.values())

print(
    f"This estimates a total of {total_entity_estimate:,} unique entities.\n"
    f"Of these, {at_least_1_mapping:,} ({at_least_1_mapping/total_entity_estimate:.1%}) have at least one mapping.\n"
    f"{only_1_mapping:,} ({only_1_mapping/total_entity_estimate:.1%}) are unique to a single resource.\n"
    f"{conserved:,} ({conserved/total_entity_estimate:.1%}) appear in all {len(CONFIGURATION.priority)} resources.\n\n"
    "This estimate is susceptible to several caveats:\n\n"
    "- Missing mappings inflates this measurement\n"
    "- Generic resources like MeSH contain irrelevant entities that can't be mapped\n"
)

In [ ]:
import upsetplot

example = upsetplot.from_memberships(*zip(*counter.most_common()))

"""Here's what the output from upsetplot.plot looks like:

{'matrix': <Axes: >,
 'shading': <Axes: >,
 'totals': <Axes: >,
 'intersections': <Axes: ylabel='Intersection size'>}
"""
plot_result = upsetplot.plot(
    example,
    # show_counts=True,
)
plot_result["intersections"].set_yscale("log")
plot_result["intersections"].set_ylim([1, plot_result["intersections"].get_ylim()[1]])
# plot_result["totals"].set_xlabel("Size")
mm, _ = plot_result["totals"].get_xlim()
plot_result["totals"].set_xlim([mm, 1])
plot_result["totals"].set_xscale("log")  # gets domain error

plt.tight_layout()
plt.savefig("disease_upset.svg")
plt.show()

In [ ]:
vv = Counter()
for k, v in counter.items():
    vv[len(k)] += v

max_value = max(vv.values())

fig, ax = plt.subplots(figsize=(4, 2.7))
sns.barplot(vv, ax=ax)

for index, value in vv.items():
    plt.text(index - 1, value + 0.2, f"{value:,}\n({value/total_entity_estimate:.1%})", ha="center", va="bottom")

ax.set_xlabel("# Resources a Concept Appears in")
ax.set_ylabel("Count")
ax.set_title(f"Landscape of {total_entity_estimate:,}\nUnique Concepts")
ax.set_yscale("log")
a, b = ax.get_ylim()
# since we're in a log scale, pad half of the max value to the top to make sure
# the counts fit in the box
ax.set_ylim([a, b + 1.7 * max_value])
plt.tight_layout()
plt.savefig("disease_histogram.svg")
plt.show()